In [40]:
# get api key from your .env file
# pip3 install python-dotenv

#import os
#from dotenv import load_dotenv

#load_dotenv()
#API_KEY = os.getenv('NASDAQ_API_KEY')

#print(str(API_KEY))
API_KEY = 'J59yaPXAwxro4VBa2ifr'

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the requests package, which can be easily downloaded using pip or conda. You can find the documentation for the package here: http://docs.python-requests.org/en/master/

Finally, apart from the requests package, you are encouraged to not use any third party Python packages, such as pandas, and instead focus on what's available in the Python Standard Library (the collections module might come in handy: https://pymotw.com/3/collections/). Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists. You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [23]:
# First, import the relevant modules
import requests 
import collections

In [42]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2014-01-01&end_date=2014-01-02&api_key="+API_KEY
data = requests.get(url)
data.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2014-01-01',
  'end_date': '2014-01-02',
  'frequency': 'daily',
  'data': [['2014-01-02',
    24.24,
    24.76,
    23.77,
    24.18,
    None,
    55735.0,
    1355658.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [52]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
#(keep in mind that the date format is YYYY-MM-DD).

data = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2018-01-01&api_key="+API_KEY)

In [45]:
# 2. Convert the returned JSON object into a Python dictionary.
AFX_dict = data.json()
type(AFX_dict)

dict

In [46]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
opens = []

for data in AFX_dict['dataset_data']['data']: 
    if type(data[1]) == float:
        opens.append(data[1])

high_open = max(opens)
low_open = min(opens)
print('The highest opening price was: $' + str(high_open))
print('The lowest opening price was: $' + str(low_open))
print(len(opens))

The highest opening price was: $53.11
The lowest opening price was: $34.0
252


In [47]:
# 4. What was the largest change in any one day (based on High and Low price)?
price_changes = []

for data in AFX_dict['dataset_data']['data']: 
    if type(data[2]) == float and type(data[3]) == float:
        change = data[2] - data[3]
        price_changes.append(change)

print('The largest change in any one day: $' + str(max(price_changes)))
print(len(price_changes))

The largest change in any one day: $2.8100000000000023
255


In [48]:
# 5. What was the largest change between any two days (based on Closing Price)?
dates = []
opens = []
closes = []
date_open_close = []

for i, data in enumerate(reversed(AFX_dict['dataset_data']['data'])): 
    #gotta make a separate date, open, close array, and then do the i and i-1 analysis
    
    if type(data[1]) == float and type(data[4]) == float:
        dates.append(data[0])
        opens.append(data[1])
        closes.append(data[4])

date_open_close = [dates, opens, closes]

In [49]:
daily_changes = []

for i in range(len(dates)):
    if i != 0:
        d_change = opens[i] - closes[i-1]
        daily_changes.append(d_change)
        
print('The largest change between any two days was: $' + str(max(daily_changes)))

The largest change between any two days was: $0.7800000000000011


In [50]:
# 6. What was the average daily trading volume during this year?
traded_volumes = []

for data in AFX_dict['dataset_data']['data']: 
    if type(data[6]) == float or type(data[6]) == int:
        traded_volumes.append(data[6])
        
avg = sum(traded_volumes) / len(traded_volumes)
print('The average daily trading volume during this year (traded_volumes): ' + str(avg))

The average daily trading volume during this year (traded_volumes): 89124.33725490196


In [51]:
# 7. (Optional) What was the median trading volume during this year. 
#(Note: you may need to implement your own function for calculating the median.)

s_tv = sorted(traded_volumes)
half = len(s_tv)/2 
median = s_tv[int(half)]
print('The median for the traded volume is: ' + str(median))

The median for the traded volume is: 76286.0
